In [7]:
import numpy as np

def generate_qpsk(K, num_samples):
    # QPSK: {±1 ± j} / sqrt(2)
    const = np.array([1+1j, 1-1j, -1+1j, -1-1j]) / np.sqrt(2)
    idx = np.random.randint(0, 4, size=(num_samples, K))
    return const[idx]  # (num_samples, K)

def one_bit_dac(x, eta):
    # x: (..., )
    xr = np.sign(np.real(x))
    xi = np.sign(np.imag(x))
    xr[xr == 0] = 1
    xi[xi == 0] = 1
    return np.sqrt(eta/2.0) * (xr + 1j*xi)

def simulate_quant_error_corr(L=32, K=1, num_samples=40000):
    """
    L: AP 数（= fronthaul DAC 数）
    K: 用户数（= data streams 数）
    num_samples: Monte Carlo 样本数
    """
    # Rayleigh 信道 H: L x K
    H = (np.random.randn(L, K) + 1j*np.random.randn(L, K)) / np.sqrt(2)
    W = np.conj(H)  # MRT / CB 预编码

    # 生成数据符号
    s = generate_qpsk(K, num_samples)    # (num_samples, K)

    # 预编码信号 x: (num_samples, L)
    x = s @ W.T

    # 对每个样本做归一化，使每次发射总功率约为 1
    power = np.mean(np.abs(x)**2, axis=1, keepdims=True)
    x = x / np.sqrt(power + 1e-12)

    # 1-bit DAC 量化（无 dithering）
    eta = 1.0 / L
    x_q = one_bit_dac(x, eta)

    # 量化失真
    e = x_q - x

    # 估计相关系数矩阵
    e_center = e - np.mean(e, axis=0, keepdims=True)
    cov = e_center.conj().T @ e_center / (num_samples - 1)   # L x L
    var = np.real(np.diag(cov))
    std = np.sqrt(var + 1e-15)
    corr = cov / (std[:, None] * std[None, :])               # 相关系数矩阵

    # 统计平均绝对相关系数（只看非对角）
    Lh = corr.shape[0]
    off = corr.copy()
    off[np.eye(Lh, dtype=bool)] = 0
    avg_abs = np.mean(np.abs(off[np.triu_indices(Lh, k=1)]))
    max_abs = np.max(np.abs(off[np.triu_indices(Lh, k=1)]))

    return corr, avg_abs, max_abs

if __name__ == "__main__":
    for K in [1, 2, 4, 8, 16, 32]:
        corr, avg_abs, max_abs = simulate_quant_error_corr(L=32, K=K, num_samples=40000)
        print(f"K = {K}: avg |corr(off-diag)| = {avg_abs:.3f}, max |corr| = {max_abs:.3f}")


K = 1: avg |corr(off-diag)| = 1.000, max |corr| = 1.000
K = 2: avg |corr(off-diag)| = 0.649, max |corr| = 0.993
K = 4: avg |corr(off-diag)| = 0.439, max |corr| = 0.936
K = 8: avg |corr(off-diag)| = 0.303, max |corr| = 0.867
K = 16: avg |corr(off-diag)| = 0.213, max |corr| = 0.564
K = 32: avg |corr(off-diag)| = 0.145, max |corr| = 0.404
